In [1]:
!pip install bs4

In [18]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
resp = requests.get('https://quotes.toscrape.com/')

resp.text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>\n        <sp

In [3]:
# Soup will help us to parse the HTML content

soup = BeautifulSoup(resp.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="

In [4]:
# To Scrape the first quote
soup.find('span', class_ = 'text').text

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [5]:
# Scrapping all the quotes
quotes = soup.find_all('span', class_ = 'text')

for quote in quotes:
    print(quote.text)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”


## Scrapping the Autor Name and the Tags

In [10]:
# Scrapping all the details
data = []

quotes = soup.find_all('div', class_ = 'quote')

for sp in quotes:
    quote = sp.find('span', class_ = 'text').text
    author = sp.find('small', class_ = 'author').text
    author_id = sp.find('a').get('href')
    tags = []
    for tag in sp.find_all('a', class_ = 'tag'):
        tags.append(tag.text)
    tags = ', '.join(tags)
    data.append([quote, author, author_id, tags])

In [11]:
data[0]

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 'Albert Einstein',
 '/author/Albert-Einstein',
 'change, deep-thoughts, thinking, world']

In [14]:
pd.DataFrame(data, columns = ['Quote', 'Author', 'Author ID', 'Tags'])

,Quote,Author,Author ID,Tags
0,“The world as we have created it is a process ...,Albert Einstein,/author/Albert-Einstein,"change, deep-thoughts, thinking, world"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,/author/J-K-Rowling,"abilities, choices"
2,“There are only two ways to live your life. On...,Albert Einstein,/author/Albert-Einstein,"inspirational, life, live, miracle, miracles"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,/author/Jane-Austen,"aliteracy, books, classic, humor"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,/author/Marilyn-Monroe,"be-yourself, inspirational"
5,“Try not to become a man of success. Rather be...,Albert Einstein,/author/Albert-Einstein,"adulthood, success, value"
6,“It is better to be hated for what you are tha...,André Gide,/author/Andre-Gide,"life, love"
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,/author/Thomas-A-Edison,"edison, failure, inspirational, paraphrased"
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,/author/Eleanor-Roosevelt,misattributed-eleanor-roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin,/author/Steve-Martin,"humor, obvious, simile"


In [15]:
df = pd.DataFrame(data, columns = ['Quote', 'Author', 'Author ID', 'Tags'])

In [16]:
df.to_csv('quotes.csv', index = False)

## Scraping For Multiple Pages

In [22]:
data = []
# tqdm is used to show the progress bar
for page in tqdm(range(1, 11)):
    
    link = 'https://quotes.toscrape.com/page/' + str(page) + '/'
    resp = requests.get(link)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    quotes = soup.find_all('div', class_ = 'quote')
    
    for sp in quotes:
        quote = sp.find('span', class_ = 'text').text[1:-1]
        author = sp.find('small', class_ = 'author').text
        author_id = sp.find('a').get('href')
        tags = []
        for tag in sp.find_all('a', class_ = 'tag'):
            tags.append(tag.text)
        tags = ', '.join(tags)
        data.append([quote, author, author_id, tags])

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:39<00:00,  3.95s/it]


In [23]:
df = pd.DataFrame(data, columns = ['Quote', 'Author', 'Author ID', 'Tags'])

In [24]:
df.to_csv('quotes.csv', index = False)